# MACHINE LEARNING

In [77]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
%matplotlib inline
pd.set_option("display.max_columns", None)
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

## Preprocessing the data

In [36]:
df = pd.read_csv('lol_ml_data.csv')

In [37]:
df.head()

,p1_champ,p2_champ,p3_champ,p4_champ,p5_champ,p6_champ,p7_champ,p8_champ,p9_champ,p10_champ,game_length_mins,blue_team_win,red_team_win,p1_spell1,p2_spell1,p3_spell1,p4_spell1,p5_spell1,p6_spell1,p7_spell1,p8_spell1,p9_spell1,p10_spell1,p1_spell2,p2_spell2,p3_spell2,p4_spell2,p5_spell2,p6_spell2,p7_spell2,p8_spell2,p9_spell2,p10_spell2,p1perkPrimaryStyle,p2perkPrimaryStyle,p3perkPrimaryStyle,p4perkPrimaryStyle,p5perkPrimaryStyle,p6perkPrimaryStyle,p7perkPrimaryStyle,p8perkPrimaryStyle,p9perkPrimaryStyle,p10perkPrimaryStyle,p1perkSubStyle,p2perkSubStyle,p3perkSubStyle,p4perkSubStyle,p5perkSubStyle,p6perkSubStyle,p7perkSubStyle,p8perkSubStyle,p9perkSubStyle,p10perkSubStyle,p1_champ_prim_role,p2_champ_prim_role,p3_champ_prim_role,p4_champ_prim_role,p5_champ_prim_role,p6_champ_prim_role,p7_champ_prim_role,p8_champ_prim_role,p9_champ_prim_role,p10_champ_prim_role,p1_champ_sec_role,p2_champ_sec_role,p3_champ_sec_role,p4_champ_sec_role,p5_champ_sec_role,p6_champ_sec_role,p7_champ_sec_role,p8_champ_sec_role,p9_champ_sec_role,p10_champ_sec_role,tank_count,mage_count,marksman_count,support_count,fighter_count,assassin_count
0,Karma,Teemo,JarvanIV,Ezreal,Neeko,Ryze,Kaisa,Cassiopeia,Braum,Rengar,38.466667,Win,Fail,SummonerFlash,SummonerIgnite,SummonerFlash,SummonerFlash,SummonerFlash,SummonerTeleport,SummonerFlash,SummonerBoost,SummonerIgnite,SummonerFlash,SummonerIgnite,SummonerFlash,SummonerSmite,SummonerHeal,SummonerIgnite,SummonerFlash,SummonerHeal,SummonerFlash,SummonerFlash,SummonerSmite,Sorcery,Sorcery,Domination,Inspiration,Sorcery,Sorcery,Inspiration,Sorcery,Resolve,Domination,Inspiration,Domination,Inspiration,Sorcery,Domination,Domination,Sorcery,Domination,Inspiration,Sorcery,Mage,Marksman,Tank,Marksman,Mage,Mage,Marksman,Mage,Support,Assassin,Support,Assassin,Fighter,Mage,Support,Fighter,None,None,Tank,Fighter,2,5,3,3,3,2
1,Teemo,Tristana,LeeSin,Bard,Leblanc,Lucian,Irelia,Blitzcrank,Orianna,Kindred,23.400000,Win,Fail,SummonerFlash,SummonerFlash,SummonerFlash,SummonerFlash,SummonerIgnite,SummonerHeal,SummonerTeleport,SummonerIgnite,SummonerBarrier,SummonerFlash,SummonerIgnite,SummonerHeal,SummonerSmite,SummonerIgnite,SummonerFlash,SummonerFlash,SummonerFlash,SummonerFlash,SummonerFlash,SummonerSmite,Sorcery,Inspiration,Domination,Domination,Domination,Inspiration,Inspiration,Resolve,Sorcery,Inspiration,Resolve,Sorcery,Inspiration,Resolve,Sorcery,Sorcery,Sorcery,Inspiration,Inspiration,Domination,Marksman,Marksman,Fighter,Support,Assassin,Marksman,Fighter,Tank,Mage,Marksman,Assassin,Assassin,Assassin,Mage,Mage,None,Assassin,Fighter,Support,None,1,3,4,2,3,5
2,JarvanIV,Lux,Zyra,Jax,Kaisa,Tryndamere,Ezreal,Thresh,Xerath,Karthus,31.733333,Fail,Win,SummonerFlash,SummonerFlash,SummonerTeleport,SummonerTeleport,SummonerFlash,SummonerIgnite,SummonerHeal,SummonerIgnite,SummonerHeal,SummonerSmite,SummonerSmite,SummonerIgnite,SummonerFlash,SummonerFlash,SummonerHeal,SummonerFlash,SummonerFlash,SummonerFlash,SummonerFlash,SummonerFlash,Domination,Sorcery,Sorcery,Inspiration,Inspiration,Inspiration,Inspiration,Resolve,Sorcery,Domination,Inspiration,Inspiration,Domination,Inspiration,Sorcery,Resolve,Sorcery,Inspiration,Inspiration,Inspiration,Tank,Mage,Mage,Fighter,Marksman,Fighter,Marksman,Support,Mage,Mage,Fighter,Support,Support,Assassin,None,Assassin,Mage,Fighter,Assassin,None,1,5,2,3,4,3
3,Blitzcrank,Graves,Caitlyn,Kled,Diana,LeeSin,Lucian,Thresh,Lissandra,Urgot,35.366667,Win,Fail,SummonerIgnite,SummonerFlash,SummonerHeal,SummonerTeleport,SummonerFlash,SummonerFlash,SummonerHeal,SummonerFlash,SummonerTeleport,SummonerFlash,SummonerFlash,SummonerSmite,SummonerFlash,SummonerFlash,SummonerTeleport,SummonerSmite,SummonerFlash,SummonerIgnite,SummonerFlash,SummonerTeleport,Resolve,Inspiration,Inspiration,Inspiration,Sorcery,Domination,Inspiration,Resolve,Sorcery,Inspiration,Sorcery,Sorcery,Sorcery,Resolve,Domination,Inspiration,Sorcery,Inspiration,Inspiration,Resolve,Tank,Marksman,Marksman,Fighter,Fighter,Fighter,Marksman,Support,Mage,Fighter

In [40]:
#the data we want to predict
red_win = df['red_team_win'].values
blue_win = df['blue_team_win'].values
game_length = df['game_length_mins'].values

In [71]:
red_win[:5]

array(['Fail', 'Fail', 'Win', 'Fail', 'Win'], dtype=object)

In [52]:
#Create features
X = df.drop(['red_team_win', 'blue_team_win', 'game_length_mins'], axis=1)

In [53]:
#Create a list of the categorical features
col_list = X.columns.tolist()
cat_feat = col_list[:-6]

In [54]:
#Create dummy variables for categorical features
X = pd.get_dummies(X, columns=cat_feat, drop_first=True)

## Building the models

### Let's test a Random Forest Classifier out of the box with out hyper parameter tuning.

In [61]:
rf = RandomForestClassifier()
X_train, X_test, y_train, y_test = train_test_split(X, red_win, test_size=0.2, random_state=42)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

### Get the accuracy score for our Random Forest model.

In [63]:
rf.score(X_test, y_test)

0.8074592890912274

### Cross Validation for the Random Forest model.

In [84]:
rf2 = RandomForestClassifier()
cv_rf_results = cross_val_score(rf2, X, red_win, cv=5)
print(cv_rf_results)

[0.92365878 0.95885134 0.86366183 0.65612524 0.67160468]


In [85]:
# Get the mean score of the cross validation
np.mean(cv_rf_results)

0.8147803728451987

### Try out of the box Random Forest Model again and scaling the features

In [86]:
# Set up pipeline
steps = [('scaler', StandardScaler()),
        ('rforest', RandomForestClassifier())]

pl = Pipeline(steps)

In [87]:
#Cross validation
scaled_cv_rf_results = cross_val_score(pl, X, red_win, cv=5)

In [88]:
print(scaled_cv_rf_results)

[0.92505953 0.95766065 0.86124536 0.6562303  0.67412622]


In [89]:
np.mean(scaled_cv_rf_results)

0.8148644128337945

By scaling the features, we get slightly better performance from the model.